In [ ]:
import os
import sys
import random
import pickle
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as la
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import directed_hausdorff

sys.path.insert(0, '../../../methods')
from sorte import *

In [ ]:
snr_list = [-20,-15,-10,-5,0,5,10,15,20,25,30]
T_list = [100,200,500,1000,2000,5000,10000]
rmse_list1 = []
rmse_list2 = []
hausdorff_list1 = []
hausdorff_list2 = []
pdet_list1 = []
pdet_list2 = []
sn_acc_list1 = []
sn_acc_list2 = []

for snr in snr_list:
    # load data
    data = np.load(f'../../../data/experiment_3/scenario_1/data_test_snr{snr}_t1000.npy', allow_pickle=True)

    # get labels and data seperately
    truths = [s['label'] for s in data]
    sensor_pos = [s['sensor_pos'] for s in data]
    data = [s['cm'] for s in data]

    # apply music algorithm
    res = 1
    N_max = 4
    p_list = []
    h_list = []
    a_list = []
    n_list = []

    for i in range(len(data)):
        R = data[i]
        M = R.shape[0]
        N = truths[i].shape[0]
        truths[i] = truths[i].reshape(-1,)

        preds, spectrum = music(R,M,N_max,1000,sensor_pos[i],res)
        preds = np.sort(preds)
        truths[i] = np.sort(truths[i])
        
        p_list.append(preds)

        hausdorff1 = directed_hausdorff(preds.reshape(-1,1), truths[i].reshape(-1,1))[0]
        hausdorff2 = directed_hausdorff(truths[i].reshape(-1,1), preds.reshape(-1,1))[0]
        h_list.append(max(hausdorff1, hausdorff2))
        
        # probability of detection
        temp = np.zeros(N)
        temp[:len(preds)] = 1
        a_list.extend(temp)
        
        # source number estimation accuracy
        n_list.append(N==len(preds))

        if snr==10 and i==0:
            np.save(f'../../../results/experiment_3/scenario_1/music_sorte_spectrum_{truths[0]}deg.npy', spectrum)

    hausdorff = np.mean(h_list)
    pdet = np.sum(a_list) / len(a_list)
    sn_acc = np.sum(n_list) / len(n_list)
    hausdorff_list1.append(hausdorff)
    pdet_list1.append(pdet)
    sn_acc_list1.append(sn_acc)
    
    print(f"snr {snr}dB, T {1000}, test-hausdorff {hausdorff:.4f}, test-prob-detection {pdet:.4f}, test-sn-acc {sn_acc:.4f}")

    with open(f'../../../results/experiment_3/scenario_1/music_sorte_preds_snr{snr}_t1000', "wb") as fp: 
        pickle.dump(p_list, fp)
    with open(f'../../../results/experiment_3/scenario_1/music_sorte_truths_snr{snr}_t1000', "wb") as fp: 
        pickle.dump(truths, fp)
    
    
for T in T_list:
    # load data
    data = np.load(f'../../../data/experiment_3/scenario_1/data_test_snr-10_t{T}.npy', allow_pickle=True)

    # get labels and data seperately
    truths = [s['label'] for s in data]
    sensor_pos = [s['sensor_pos'] for s in data]
    data = [s['cm'] for s in data]

    # apply music algorithm
    res = 1
    N_max = 4
    p_list = []
    h_list = []
    a_list = []
    n_list = []

    for i in range(len(data)):
        R = data[i]
        M = R.shape[0]
        N = truths[i].shape[0]
        truths[i] = truths[i].reshape(-1,)

        preds, spectrum = music(R,M,N_max,T,sensor_pos[i],res)
        preds = np.sort(preds)
        truths[i] = np.sort(truths[i])
        p_list.append(preds)

        hausdorff1 = directed_hausdorff(preds.reshape(-1,1), truths[i].reshape(-1,1))[0]
        hausdorff2 = directed_hausdorff(truths[i].reshape(-1,1), preds.reshape(-1,1))[0]
        h_list.append(max(hausdorff1, hausdorff2))
        
        # probability of detection
        temp = np.zeros(N)
        temp[:len(preds)] = 1
        a_list.extend(temp)
        
        # source number estimation accuracy
        n_list.append(N==len(preds))
    
    hausdorff = np.mean(h_list)
    pdet = np.sum(a_list) / len(a_list)
    sn_acc = np.sum(n_list) / len(n_list)
    hausdorff_list2.append(hausdorff)
    pdet_list2.append(pdet)
    sn_acc_list2.append(sn_acc)
    
    print(f"snr -10 dB, T {T}, test-hausdorff {hausdorff:.4f}, test-prob-detection {pdet:.4f}, test-sn-acc {sn_acc:.4f}")

    with open(f'../../../results/experiment_3/scenario_1/music_sorte_preds_snr-10_t{T}', "wb") as fp: 
        pickle.dump(p_list, fp)
    with open(f'../../../results/experiment_3/scenario_1/music_sorte_truths_snr-10_t{T}', "wb") as fp: 
        pickle.dump(truths, fp)
    
np.save('../../../results/experiment_3/scenario_1/music_sorte_hausdorff1.npy', hausdorff_list1)
np.save('../../../results/experiment_3/scenario_1/music_sorte_hausdorff2.npy', hausdorff_list2)
np.save('../../../results/experiment_3/scenario_1/music_sorte_pdet1.npy', pdet_list1)
np.save('../../../results/experiment_3/scenario_1/music_sorte_pdet2.npy', pdet_list2)
np.save('../../../results/experiment_3/scenario_1/music_sorte_snacc1.npy', sn_acc_list1)
np.save('../../../results/experiment_3/scenario_1/music_sorte_snacc2.npy', sn_acc_list2)

In [ ]:
# plot rmse values
plt.figure()
plt.plot(snr_list, hausdorff_list1, '-o')
plt.title("rmse values for different snr levels")
plt.xlabel("snr (dB)")
plt.ylabel("hausdorff distance")
plt.legend(['music+sorte'])
plt.yscale("log")
plt.grid()

plt.figure()
plt.plot(T_list, hausdorff_list2, '-o')
plt.title("rmse values for different snapshot numbers")
plt.xlabel("T")
plt.ylabel("hausdorff distance")
plt.legend(['music+sorte'])
plt.yscale("log")
plt.grid()